###  KNN_Отзывы о продуктах питания и гурманской пище file_path = 'Grocery_and_Gourmet_Food_5

- Рекомендательные системы — это комплекс сервисов и программ, который анализирует предпочтения пользователей и пытается предсказать, что может их заинтересовать. Они помогают клиентам определиться с выбором товара или контента, а также повышают лояльность, конверсию и средний чек интернет-магазинов, предприятий быстрого питания, телекоммуникационных и финансовых организаций.
- Существует два основных типа рекомендательных систем: контентно-ориентированная и коллаборативная фильтрация.
- Контентно-ориентированная фильтрация основана на анализе характеристик товаров и предпочтениях пользователя. Система анализирует содержание товаров, такие как текстовые описания, категории, ключевые слова и т. д., и сопоставляет их с предпочтениями пользователя. На основе этого анализа система предлагает рекомендации, соответствующие интересам пользователя.
- Коллаборативная фильтрация рекомендует продукты как на основе оценок самого покупателя, так и на основе явных и неявных предпочтений других пользователей. Например, если вы купили стиральную машину, то коллаборативная система предложит товары, которыми интересовались другие покупатели стиральных машин, а не машину другого бренда.
- Универсальных методов не существует, и всё чаще применяется гибридный подход, который сочетает методы для получения максимально точных рекомендаций на имеющихся данных и при минимальных затратах времени.

Проведите базовый EDA - распределения рейтингов, количество уникальных товаров и т.д.
Отложите часть данных для тестирования. В рекомендательных системах для этого можно случайным образом “занулить” желаемый процент рейтингов в исходном датасете, чтобы получить “тренировочный набор”, и проверять качество ваших рекомендаций на этих уже не зануленных рейтингах. При этом, если у вас есть временная зависимость в данных, имеет смысл занулять рейтинги “из будущего”, чтобы обучаться на “исторических” покупках/просмотрах и т.д.
На основании вашего датасета постройте рекомендательную систему.
Оцените качество полученных рекомендаций, при помощи подходящих метрик (если вы использовали рейтинги, можно взять RMSE).

- Скачаем набор данных Amazon, который включает отзывы пользователей о различных продуктах. Данные предоставляются в формате JSON и могут быть использованы для экспериментов и анализа данных.
- "5-core" и "ratings only" относятся к двум разным типам данных, доступных на сайте Amazon Review Data.
- "5-core" относится к подмножеству данных, в котором каждое из оставшихся пользователей и элементов имеет 5 отзывов каждый. Это означает, что если у пользователя было менее 5 отзывов, он был исключен из этого набора данных. Эти данные обычно используются для экспериментов, где требуется большое количество отзывов от каждого пользователя и каждого товара.
- "ratings only" относится к набору данных, который включает только кортежи (item, user, rating, timestamp). 
- Каждый поднабор включает информацию об отзывах пользователей, такую как рейтинг, время отзыва, идентификатор отзывателя и текст отзыва. Это означает, что в этих данных нет метаданных или самих отзывов, они включают только рейтинги. 

Такие данные обычно используются для обучения рекомендательных систем, так как они содержат только необходимые данные для предсказания рейтингов.
- Ура, нужно обучить рекомендательную систему, то "ratings only" может быть более подходящим.

Один из способов использования этих данных - это создание системы рекомендаций. Например, вы можете использовать модель матричного факторизации для предсказания рейтингов на основе только рейтинговых данных. Это может быть особенно полезно для компаний, которые хотят улучшить свои системы рекомендаций и увеличить продажи.

Другой возможный подход - это анализ отзывов для определения общего мнения клиентов о конкретных продуктах или категориях продуктов. Это может помочь компаниям улучшить свои продукты или изменить свою стратегию маркетинга.

### KNN

#### Импорт библиотек

In [ ]:
# Установка ширины контейнера для отображения
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [4]:
from typing import List, Dict # Импортируем типы List и Dict из модуля typing

In [5]:
# Импорт класса TfidfVectorizer из модуля feature_extraction.text
from sklearn.feature_extraction.text import TfidfVectorizer 

# Импорт функции cosine_similarity из модуля metrics.pairwise
from sklearn.metrics.pairwise import cosine_similarity 

In [38]:
# Импортируем необходимые библиотеки
import pandas as pd # Используется для работы с данными
import seaborn as sns
import matplotlib.pyplot as plt 
import scipy.sparse as sparse # Используется для работы с разреженными матрицами
import numpy as np # Используется для математических операций
from scipy.sparse.linalg import spsolve # Функция для решения систем линейных уравнений

In [ ]:
# Импортируем необходимые библиотеки по построению модели
from sklearn.model_selection import train_test_split  # Для разделения данных на обучающую и тестовую выборки
from sklearn.metrics import mean_squared_error  # Для вычисления среднеквадратичной ошибки
from math import sqrt  # Для вычисления квадратного корня
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer  # Для преобразования текста в векторы TF-IDF
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity  # Для вычисления линейного ядра и косинусного сходства
from sklearn.neighbors import NearestNeighbors  # Для построения модели ближайших соседей

 - Как видно из описания, в этом наборе данных Grocery and Gourmet Food
- Бакалея и еда для гурманов	содержатся Метаданные, такие как текст отзыва или описание товара, включены в набор данных "Office Products" в разделе "5-core". Этот набор данных содержит полные отзывы пользователей, включая текст отзыва, рейтинг, время отзыва и другие детали.
- "Бакалея и еда для гурманов" относится к категории товаров на Amazon, которые включают в себя различные виды продуктов питания и напитков. Эта категория может включать в себя все, от сыра и хлеба до специализированных продуктов питания и напитков.

#### Загружаем датасет

In [ ]:
import pandas as pd

# Путь к файлу с отзывами о продуктах питания и гурманской пище
file_path = 'Grocery_and_Gourmet_Food_5.json.gz'  

# Чтение JSON файла по частям с использованием chunksize
chunk_size =  10000  # Размер части данных для чтения за один раз
chunks = []

for chunk in pd.read_json(file_path, lines=True, chunksize=chunk_size):
    chunks.append(chunk)

# Объединение всех частей в один DataFrame
retail_data = pd.concat(chunks, ignore_index=True)

# Вывод первых пяти строк DataFrame
print(retail_data.head())


 - Используем функцию pd.read_json из библиотеки pandas для чтения больших JSON файлов по частям (или "чанкам"), что позволяет обрабатывать данные в удобных порциях, не загружая всё файл в память сразу. Это особенно полезно при работе с большими наборами данных.

#### Проводим предпроверочный анализ

In [ ]:
# Вывод первых пяти строк данных
retail_data.head()

- Используем библиотеку pandas для чтения данных из сжатого JSON файла с отзывами о продуктах питания и гурманской пище. 
- ниже представлен перевод на русский язык:
- overall — общий рейтинг товара.
- verified — флаг подтверждения отзыва.
- reviewTime — время отзыва в читаемом формате.
- reviewerID — идентификатор пользователя-автора отзыва.
- asin — идентификатор товара Amazon Standard Identification Number.
- reviewerName — имя пользователя-автора отзыва.
- reviewText — текст отзыва.
- summary — краткое содержание отзыва.
- unixReviewTime — время отзыва в формате Unix timestamp.
- vote — количество полезных голосов для отзыва.
- style — метаданные товара (например, формат, цвет и т.д.).
- image — изображения, добавленные пользователем после получения товара.

In [ ]:
# Смотрим на размерность
retail_data.shape

### Препроцессинг

In [ ]:
# Анализируем колонку
# Статистика по рейтингам
print(retail_data['overall'].describe())

In [ ]:
# Анализируем колонку
retail_data['summary'].value_counts()

In [ ]:
# Распределение рейтингов
sns.countplot(x='overall', data=retail_data)
plt.title('Распределение рейтингов')
plt.xlabel('Рейтинг')
plt.ylabel('Количество')
plt.show()

In [ ]:
# Количество уникальных товаров
retail_data['asin'].nunique()

In [ ]:
# Работа с пропущенными значениями
retail_data.isnull().sum()

In [ ]:
# Удаление строк с пропущенными значениями
retail_data = retail_data.dropna()

In [ ]:
# Работа с пропущенными значениями
retail_data.isnull().sum()

Я хотел оставить колонки 
Grocery and Gourmet Food ratings only (5,074,160 ratings), названия колонок asin, reviewerID, unixReviewTime, overall и удалить оставшиеся из 5-core (1,143,860 reviews).
Однако подумал, нужно провести анализ

- Наш набор данных, вероятно, содержит категориальные признаки или текстовые данные, которые не могут быть напрямую использованы в модели RandomForestRegressor.
- Нам нужно предварительно обработать эти признаки, прежде чем использовать их в модели. Например, мы можем использовать кодирование с помощью OneHotEncoder или LabelEncoder из sklearn.preprocessing для категориальных признаков. 
- Для текстовых данных мы можете использовать методы векторизации текста, такие как CountVectorizer или TfidfVectorizer.

In [ ]:
retail_data.info()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

# Создаем копии retail_data для анализа важности признаков
retail_data_copy = retail_data.copy()

In [ ]:
# Отложим часть данных для тестирования
train_data, test_data = train_test_split(retail_data_copy, test_size=0.2, random_state=42) 
# Разделяем данные на обучающую и тестовую выборки

In [ ]:
# Определение признаков для обучения
# Оставляем только числовые признаки
numeric_features = train_data.select_dtypes(include=[np.number])
features = numeric_features.drop('overall', axis=1)
target = numeric_features['overall']

In [ ]:
# Создание модели RandomForestRegressor
model = RandomForestRegressor()

In [ ]:
# Обучение модели на тренировочных данных
model.fit(features, target)

In [ ]:
# Вычисление важности признаков
feature_importances = model.feature_importances_

# Создание DataFrame для важности признаков
feature_importances_df = pd.DataFrame({
    'feature': features.columns,
    'importance': feature_importances
})

# Сортируем по важности признаков
feature_importances_df.sort_values('importance', ascending=False, inplace=True)

# Выводим важность признаков
print(feature_importances_df)

Вывод:
- В контексте рекомендательных систем, основанных на методе ближайших соседей (KNN), важность признака unixReviewTime может быть не так очевидна.
- Метод KNN обычно используется для коллаборативной фильтрации, где рекомендации генерируются на основе сходства между пользователями или элементами. Это сходство обычно определяется на основе рейтингов, которые пользователи дают элементам, а не на основе других признаков, таких как unixReviewTime.
- Однако, если мы используем метод KNN в контексте контентной фильтрации, где рекомендации генерируются на основе сходства между признаками элементов, тогда unixReviewTime может иметь значение. Например, если элементы, которые были оценены в более ранний период времени, считаются менее релевантными для пользователя, то unixReviewTime может быть важным признаком.
- Важность признака зависит от конкретной модели и данных, и может быть разной в разных контекстах. Важность признака не говорит о том, как именно этот признак влияет на прогнозы. Это просто показывает, что признак unixReviewTime имеет значительное влияние на прогнозы модели.

Удаляем названия колонок, которые нам не нужны в контексте обучения рекомендательных систем

In [ ]:
# Удаляем столбцы из DataFrame
retail_data = retail_data_copy.drop(['verified', 'reviewTime', 'reviewerName',
                                     'reviewText', 'summary', 'vote', 'style', 'image'], axis=1)


In [26]:
retail_data.columns

Index(['overall', 'reviewerID', 'asin', 'unixReviewTime'], dtype='object')

In [27]:
retail_data.head()

,overall,reviewerID,asin,unixReviewTime
393,4,A2R0EBFJFCNSZM,B00006FWVX,1462579200
578,5,A1QC1C545QHNZG,B00008RCN8,1447891200
873,3,ALJ77PYM7D513,B0000CD02T,1312502400
961,5,A1Q5AHXG0HVV34,B0000CFH7B,1502582400
978,4,A21DSKR8PDX0ZY,B0000CFH7B,1466035200


- overall — общий рейтинг товара.
- reviewerID — идентификатор пользователя-автора отзыва.
- asin — идентификатор товара Amazon Standard Identification Number.
- unixReviewTime — время отзыва в формате Unix timestamp.

#### Моделирование

In [28]:
# Отложим часть данных для тестирования
train_data, test_data = train_test_split(retail_data, test_size=0.2, random_state=42) 
# test_size =  0.2  20% данных будет использоваться для тестирования
# random_state =  42  Устанавливаем seed для генератора случайных чисел для воспроизводимости результатов
# Разделяем данные на обучающую и тестовую выборки

In [29]:
# Создаем маску, которая будет использоваться для выбора случайного подмножества индексов
# Здесь  0.8 - это процент рейтингов, которые мы хотим занулить
mask = np.random.rand(len(train_data)) <  0.8  # Генерируем случайные числа в диапазоне [0,  1) и сравниваем их с  0.8

# Используем маску для выбора индексов, которые нужно занулить
zero_indices = train_data[mask].index  # Получаем индексы строк, где маска равна True

# Зануляем выбранные рейтинги в колонке 'overall'
train_data.loc[zero_indices, 'overall'] =  0  # Используем метод .loc для доступа к строкам по индексам и установки значения в  0

- 0.8 обозначает процент рейтингов, которые мы хотим занулить в обучающем наборе данных. Это делается для имитации ситуации, когда у нас есть некоторые пользователи и товары, но мы не знаем всех рейтингов, которые пользователи поставили товарам. Зануление некоторых рейтингов позволяет протестировать, насколько хорошо рекомендательная система может предсказать эти зануленные рейтинги.
- В данном случае, строка mask = np.random.rand(len(train_data)) < 0.8 генерирует случайные числа в диапазоне от 0 до 1 для каждого рейтинга в обучающем наборе данных. Если случайное число меньше 0.8, то соответствующий рейтинг зануляется.
- Это общепринятый подход для создания “тренировочного набора” в рекомендательных системах. Однако, важно помнить, что выбор процента зануления может влиять на результаты вашей модели, и его следует выбирать в соответствии с конкретными требованиями и ограничениями.

In [30]:
# 'train_data' - это DataFrame, который содержит данные о рейтингах пользователей
# 'reviewerID' - это столбец с идентификаторами пользователей
# 'asin' - это столбец с идентификаторами товаров
# 'overall' - это столбец с рейтингами товаров

# Создаем матрицу пользователь-товар
# В качестве индекса используем 'reviewerID', в качестве столбцов - 'asin', а в качестве значений - 'overall'
# fill_value=0 говорит о том, что если для определенного пользователя и товара нет рейтинга, то в матрице будет  0
user_item_matrix = train_data.pivot_table(index='reviewerID', columns='asin', values='overall', fill_value=0)

In [31]:
user_item_matrix.head()

asin,B00008RCN8,B0000CD02T,B0000CFH7B,B0000CFPI2,B0000D916Y,B0000DG86X,B0000DG8AR,B0000DGF9V,B0000DHXGL,B0000DID5X,...,B01H5NAW0M,B01H6IQING,B01H73VY7K,B01HATRDLC,B01HC0BJ9G,B01HCL8A1A,B01HH79XRE,B01HH7IWXU,B01HH9IN64,B01HHXJEIG
reviewerID,,,,,,,,,,,,,,,,,,,,,
A0718784TOWF9DX1DOFD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1025ZA8TGG21H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A104STHE0V2TIY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A105LNR0SU1038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10898JSSFDCQ2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# DataFrame, содержащий вашу матрицу пользователь-товар
# 'user_item_matrix' - это DataFrame, где каждая строка представляет пользователя, а каждый столбец - товар
# с рейтингами в качестве значений.

# Создаем экземпляр класса NearestNeighbors
# Мы будем использовать метрику косинусного сходства, которая подходит для разреженных данных, и алгоритм полного перебора
# Мы установим количество соседей для рассмотрения на  20 и использовать все доступные CPU для параллельных вычислений.
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
# Обучаем модель на ваших данных
# Метод fit обучает модель на матрице пользователь-товар, которая является входными данными для алгоритма ближайших соседей.
model_knn.fit(user_item_matrix)

# Теперь модель обучена и готова к предсказаниям
# Мы можем использовать модель для поиска ближайших соседей для любого заданного пользователя или товара.

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [33]:
# Получаем рекомендации для каждого пользователя
# В этом примере мы выбираем первую рекомендацию для каждого пользователя
# Применяем модель k-NN для получения рекомендаций для каждого пользователя
# Мы используем метод apply, чтобы применить функцию к каждой строке DataFrame.
# Лямбда-функция принимает строку DataFrame (row) и возвращает рекомендации для этого пользователя.
# model_knn.kneighbors - это метод, который возвращает индексы k ближайших соседей.
# return_distance=False означает, что мы хотим получить только индексы соседей, а не расстояния до них.
# [0] в конце - это индекс первого элемента в возвращаемом списке, который содержит индексы соседей.
user_item_matrix['recommendations'] = user_item_matrix.apply(lambda row: model_knn.kneighbors([row.values], 
                                                                                              return_distance=False)[0][0], axis=1)
# Теперь в каждой строке DataFrame 'user_item_matrix' есть новый столбец 'recommendations',
# который содержит индексы рекомендуемых предметов для соответствующего пользователя.

C:\Users\kolin\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\kolin\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\kolin\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\kolin\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\kolin\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.w

In [34]:
# Предсказываем рейтинги для тестовой выборки
# Мы используем метод apply, чтобы применить функцию к каждой строке DataFrame.
# Лямбда-функция принимает строку DataFrame (row) и возвращает рекомендации для этого пользователя.
# Затем мы используем эти рекомендации для получения предсказанных рейтингов из user_item_matrix.

def get_recommendations(row):
    user_id = row['reviewerID']
    if user_id in user_item_matrix.index:
        return user_item_matrix.loc[user_id, 'recommendations']
    else:
        return np.nan  # Возвращаем NaN, если пользователь не найден

test_data['pred_ratings'] = test_data.apply(get_recommendations, axis=1)

# Теперь в каждой строке DataFrame 'test_data' есть новый столбец 'pred_ratings',
# который содержит предсказанные рейтинги для рекомендованных предметов.

- Теоритическая справка
- В контексте функции apply в Pandas, axis=1 означает, что функция будет применяться к каждому столбцу DataFrame. 
- Когда мы используем axis=1, мы указываем, что функция должна быть применена к каждой строке DataFrame, а не к каждому столбцу, что является поведением по умолчанию (axis=0).

In [35]:
test_data.head()

,overall,reviewerID,asin,unixReviewTime,pred_ratings
779382,5,A3OEGVSXY0Y07W,B00GVITS62,1506816000,1206.0
78240,5,A3OEGVSXY0Y07W,B000EVMNMI,1382745600,1206.0
593019,5,A15Y86VQPRYAQD,B007P4MADE,1492732800,NaN
635066,5,A1PTCOT13F3J8V,B00AAJRPZY,1497744000,NaN
711978,5,A27H0T39U3FZB5,B00DBWUXEM,1462665600,NaN


In [36]:
# Оценим качество полученных рекомендаций
# Удаляем строки с NaN значениями перед вычислением RMSE
test_data.dropna(subset=['pred_ratings'], inplace=True)
mse = mean_squared_error(test_data['overall'], test_data['pred_ratings'])  # Вычисляем MSE
rmse = sqrt(mse)  # Вычисляем RMSE
print("RMSE:", rmse)  # Выводим RMSE

RMSE: 1041.1123530823243


#### Вывод
- Попробуем использовать - библиотеку implicit, которая является библиотекой Python, которая предоставляет реализацию алгоритма Alternating Least Squares (ALS) для рекомендательных систем.
- ALS — это метод матричного факторизации, который используется для обнаружения скрытых структур в данных и может быть применен к рекомендательным системам для предсказания предпочтений пользователей.
- RMSE (Root Mean Square Error) равный 1041.1123530823243 для нашей рекомендательной системы указывает на среднеквадратическую ошибку между истинными и предсказанными оценками рейтингов. 
- RMSE — это метрика, которая показывает, насколько хорошо модель может предсказывать значения. Чем меньше значение RMSE, тем точнее предсказания модели. В данном случае, значение RMSE указывает на относительно высокую ошибку, что может свидетельствовать о необходимости дополнительной настройки модели или использования другого подхода для предсказания рейтингов

Спасибо за внимание!